# 🎤 PIPER TTS - TRAINING ITALIANO DEFINITIVO

## 📋 Panoramica
Questo notebook addestra un modello Piper TTS in italiano usando il dataset giacomoarienti/female-LJSpeech-italian.

**Caratteristiche:**
- ✅ Training completo da zero
- ✅ Dataset: 5856 file audio (8h 23m)
- ✅ Voce femminile italiana professionale
- ✅ Sample rate: 16000Hz
- ⏱️ Tempo stimato: 12-16 ore su GPU T4

**Prerequisiti:**
- ✅ Google Colab con GPU (Runtime > Change runtime type > GPU)
- ✅ Dataset scaricato usando `Download_Dataset_Italiano_DEFINITIVO.ipynb`

---

## 🔧 STEP 1: Verifica GPU

In [ ]:
# Verifica che la GPU sia disponibile
!nvidia-smi

print("\n" + "="*60)
print("✅ Se vedi info GPU sopra, sei pronto!")
print("❌ Se vedi errore: Runtime > Change runtime type > GPU")
print("="*60)

## 📂 STEP 2: Monta Google Drive

**IMPORTANTE:** Il dataset deve essere già stato scaricato usando il notebook `Download_Dataset_Italiano_DEFINITIVO.ipynb`

In [ ]:
from google.colab import drive

print("📂 Montaggio Google Drive...\n")
drive.mount('/content/drive')
print("\n✅ Google Drive montato con successo!")

## 📦 STEP 3: Installazione Piper

In [ ]:
print("📦 Installazione Piper Training...\n")

# Installazione dipendenze base
!pip install -q onnxruntime
!pip install -q torch torchvision torchaudio

# Installazione piper-phonemize dai sorgenti (più affidabile)
print("🔧 Installazione piper-phonemize dai sorgenti...")
!apt-get install -q -y build-essential
!pip install -q pybind11

# Clona e installa piper-phonemize
!git clone https://github.com/rhasspy/piper-phonemize.git
%cd piper-phonemize
!pip install -q .
%cd /content

print("✅ piper-phonemize installato!\n")

# Clona repository Piper
print("📦 Clonazione repository Piper...")
!git clone https://github.com/rhasspy/piper.git
%cd piper/src/python

# Installa requirements (ora dovrebbe funzionare)
!pip install -q -r requirements.txt
!pip install -q -e .

%cd /content
print("\n✅ Installazione Piper completata!")

## 📁 STEP 4: Verifica Dataset

**IMPORTANTE:** Prima di eseguire questa cella, assicurati di aver scaricato il dataset usando il notebook `Download_Dataset_Italiano_DEFINITIVO.ipynb`

In [ ]:
import os
import pandas as pd
from pathlib import Path

# Path dataset (scaricato dal notebook di download)
DATASET_DIR = "/content/drive/MyDrive/ljspeech_italian"
OUTPUT_DIR = "/content/piper_output"

print("="*60)
print("  🔍 VERIFICA DATASET")
print("="*60)

# Verifica esistenza directory
if not os.path.exists(DATASET_DIR):
    print("\n❌ ERRORE: Dataset non trovato!")
    print(f"   Path cercato: {DATASET_DIR} (Google Drive)")
    print("\n💡 SOLUZIONE:")
    print("   1. Esegui prima il notebook 'Download_Dataset_Italiano_DEFINITIVO.ipynb'")
    print("   2. Assicurati che il dataset sia in Google Drive: /content/drive/MyDrive/ljspeech_italian/")
    raise FileNotFoundError(f"Dataset non trovato in {DATASET_DIR}")

# Verifica metadata.csv
metadata_path = f"{DATASET_DIR}/metadata.csv"
if not os.path.exists(metadata_path):
    raise FileNotFoundError("metadata.csv non trovato!")

# Verifica wavs/
wavs_dir = f"{DATASET_DIR}/wavs"
if not os.path.exists(wavs_dir):
    raise FileNotFoundError("Cartella wavs/ non trovata!")

# Carica e verifica metadata
metadata = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
num_metadata = len(metadata)

# Conta file WAV
wav_files = list(Path(wavs_dir).glob("*.wav"))
num_wavs = len(wav_files)

print(f"\n✅ Dataset trovato!")
print(f"   📁 Path: {DATASET_DIR}")
print(f"   📄 Trascrizioni: {num_metadata}")
print(f"   🎵 File audio: {num_wavs}")

if num_metadata == num_wavs:
    print(f"   ✅ Corrispondenza perfetta!")
else:
    print(f"   ⚠️  Warning: {num_metadata} trascrizioni vs {num_wavs} file audio")

# Crea directory output
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"\n✅ Directory output creata: {OUTPUT_DIR}")

print("\n" + "="*60)

## 🎛️ STEP 5: Configurazione Training

Qui configuriamo tutti i parametri del modello e del training.

In [ ]:
import json

print("="*60)
print("  ⚙️ CONFIGURAZIONE TRAINING")
print("="*60)

# Configurazione completa
config = {
    "audio": {
        "sample_rate": 16000,
        "max_wav_value": 32767.0,
        "filter_length": 1024,
        "hop_length": 256,
        "win_length": 1024
    },
    "model": {
        "name": "vits",
        "hidden_channels": 192,
        "inter_channels": 192,
        "filter_channels": 768,
        "n_heads": 2,
        "n_layers": 6,
        "kernel_size": 3,
        "p_dropout": 0.1
    },
    "training": {
        "epochs": 10000,
        "learning_rate": 0.0002,
        "batch_size": 16,
        "log_interval": 100,
        "save_interval": 1000,
        "num_workers": 4
    },
    "dataset": {
        "path": DATASET_DIR,
        "text_cleaners": ["basic_cleaners"],
        "language": "it-it"
    }
}

# Salva configurazione
config_path = f"{OUTPUT_DIR}/config.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Configurazione salvata: {config_path}")
print(f"\n📝 Parametri principali:")
print(f"   • Sample rate: {config['audio']['sample_rate']} Hz")
print(f"   • Lingua: {config['dataset']['language']} (Italiano)")
print(f"   • Epochs: {config['training']['epochs']:,}")
print(f"   • Batch size: {config['training']['batch_size']}")
print(f"   • Learning rate: {config['training']['learning_rate']}")
print(f"   • Salvataggio checkpoint ogni: {config['training']['save_interval']} steps")

print("\n" + "="*60)

## 🔄 STEP 6: Preprocessing Dataset

Prepara il dataset per il training (estrazione features, phonemization, etc.)

In [ ]:
print("="*60)
print("  🔄 PREPROCESSING DATASET")
print("="*60)
print("\n⏱️  Questo può richiedere 10-20 minuti...\n")

PREPROCESSED_DIR = f"{OUTPUT_DIR}/preprocessed"

!python /content/piper/src/python/piper_train/preprocess.py \
    --input-dir {DATASET_DIR} \
    --output-dir {PREPROCESSED_DIR} \
    --language it-it \
    --sample-rate 16000

print("\n" + "="*60)
print("✅ Preprocessing completato!")
print("="*60)

## 🚀 STEP 7: AVVIO TRAINING

**⚠️ ATTENZIONE:** Il training richiederà **12-16 ore** su GPU T4.

**Consigli:**
- 💡 Tieni aperta la tab del browser per evitare disconnessioni
- 💡 Il training salva checkpoint ogni 1000 step, quindi puoi riprenderlo se si interrompe
- 💡 Puoi monitorare i log per vedere i progressi

In [ ]:
print("="*60)
print("  🎯 AVVIO TRAINING")
print("="*60)
print("\n⏱️  Tempo stimato: 12-16 ore")
print("💾 Checkpoint salvati ogni 1000 step")
print("📊 Log ogni 100 step\n")
print("-"*60)

CHECKPOINT_DIR = f"{OUTPUT_DIR}/checkpoints"

%cd /content/piper/src/python

!python -m piper_train \
    --dataset-dir {PREPROCESSED_DIR} \
    --output-dir {CHECKPOINT_DIR} \
    --config {config_path} \
    --restore-checkpoint

%cd /content

print("\n" + "="*60)
print("🎉 TRAINING COMPLETATO!")
print("="*60)

## 📊 STEP 8: Verifica Checkpoint

Visualizza i checkpoint salvati durante il training.

In [ ]:
import glob

print("="*60)
print("  📊 CHECKPOINT SALVATI")
print("="*60)

checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/*.pt"))

if len(checkpoints) == 0:
    print("\n❌ Nessun checkpoint trovato!")
else:
    print(f"\n✅ Trovati {len(checkpoints)} checkpoint:\n")
    
    for ckpt in checkpoints[-10:]:  # Mostra ultimi 10
        size_mb = os.path.getsize(ckpt) / (1024*1024)
        print(f"   📁 {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
    
    if len(checkpoints) > 10:
        print(f"\n   ... e altri {len(checkpoints)-10} checkpoint")

print("\n" + "="*60)

## 🎵 STEP 9: Export Modello ONNX

Converte il checkpoint PyTorch in formato ONNX (richiesto da Piper per l'inferenza).

In [ ]:
import glob

print("="*60)
print("  📦 EXPORT MODELLO ONNX")
print("="*60)

# Trova ultimo checkpoint
checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/*.pt"))

if len(checkpoints) == 0:
    print("\n❌ Nessun checkpoint trovato!")
    print("Assicurati che il training sia completato.")
else:
    latest_checkpoint = checkpoints[-1]
    print(f"\n📁 Usando checkpoint: {os.path.basename(latest_checkpoint)}")
    
    model_output = f"{OUTPUT_DIR}/model.onnx"
    
    %cd /content/piper/src/python
    
    !python -m piper_train.export_onnx \
        {latest_checkpoint} \
        {model_output}
    
    %cd /content
    
    if os.path.exists(model_output):
        size_mb = os.path.getsize(model_output) / (1024*1024)
        print(f"\n✅ Modello esportato con successo!")
        print(f"   📁 Path: {model_output}")
        print(f"   💾 Dimensione: {size_mb:.1f} MB")
    else:
        print("\n❌ Errore durante l'export!")

print("\n" + "="*60)

## 🧪 STEP 10: Test Modello

Prova il modello generando audio da testo.

In [ ]:
from IPython.display import Audio, display
import subprocess
import os

print("="*60)
print("  🧪 TEST MODELLO")
print("="*60)

# Testi di test in italiano
test_texts = [
    "Buongiorno, questo è un test del modello Piper addestrato in italiano.",
    "La sintesi vocale funziona correttamente con frasi più lunghe e complesse.",
    "I Malavoglia è un romanzo di Giovanni Verga."
]

model_path = f"{OUTPUT_DIR}/model.onnx"

if not os.path.exists(model_path):
    print("\n❌ Modello non trovato! Esegui prima lo STEP 8.")
else:
    print("\n🎤 Generazione audio di test...\n")
    
    for i, text in enumerate(test_texts, 1):
        output_wav = f"{OUTPUT_DIR}/test_{i}.wav"
        
        print(f"{i}. Testo: \"{text}\"")
        
        # Genera audio con Piper
        result = subprocess.run(
            ["piper", "--model", model_path, "--output_file", output_wav],
            input=text.encode('utf-8'),
            capture_output=True
        )
        
        if os.path.exists(output_wav):
            print(f"   ✅ Audio generato: {output_wav}")
            display(Audio(output_wav))
            print()
        else:
            print(f"   ❌ Errore nella generazione")
            if result.stderr:
                print(f"   Errore: {result.stderr.decode('utf-8')}")

print("="*60)

## 💾 STEP 11: Download Modello

Scarica il modello finale sul tuo computer.

In [ ]:
import shutil
from google.colab import files

print("="*60)
print("  💾 DOWNLOAD MODELLO")
print("="*60)

model_path = f"{OUTPUT_DIR}/model.onnx"

if not os.path.exists(model_path):
    print("\n❌ Modello non trovato!")
else:
    print("\n📦 Creazione archivio per il download...\n")
    
    # Crea directory per il package
    package_dir = f"{OUTPUT_DIR}/piper_model_italiano"
    os.makedirs(package_dir, exist_ok=True)
    
    # Copia file necessari
    shutil.copy(model_path, package_dir)
    shutil.copy(config_path, package_dir)
    
    # Crea README
    readme_content = f"""# Piper TTS - Modello Italiano

## Informazioni Modello
- **Lingua**: Italiano (it-it)
- **Voce**: Femminile
- **Dataset**: giacomoarienti/female-LJSpeech-italian
- **Sample Rate**: 16000 Hz
- **Ore di training**: ~8h 23m di audio

## Utilizzo
```bash
echo "Ciao, questo è un test" | piper --model model.onnx --output_file output.wav
```

## File Inclusi
- `model.onnx`: Modello ONNX per l'inferenza
- `config.json`: Configurazione del modello
"""
    
    with open(f"{package_dir}/README.md", 'w', encoding='utf-8') as f:
        f.write(readme_content)
    
    # Crea ZIP
    archive_path = shutil.make_archive(
        f"{OUTPUT_DIR}/piper_model_italiano",
        'zip',
        package_dir
    )
    
    archive_size_mb = os.path.getsize(archive_path) / (1024*1024)
    
    print(f"✅ Archivio creato:")
    print(f"   📁 {os.path.basename(archive_path)}")
    print(f"   💾 Dimensione: {archive_size_mb:.1f} MB")
    print(f"\n📥 Download in corso...\n")
    
    # Download
    files.download(archive_path)
    
    print("\n✅ Download completato!")

print("\n" + "="*60)

## 🎉 COMPLETATO!

### Risultati:
- ✅ Modello Piper TTS italiano addestrato
- ✅ File ONNX pronto per l'uso
- ✅ Configurazione salvata

### Prossimi Passi:
1. **Testa il modello** con frasi diverse
2. **Fine-tuning** se necessario con più dati
3. **Integra** in applicazioni usando Piper CLI

### Utilizzo del Modello:
```bash
# Sintesi da testo
echo "Buongiorno, come stai?" | piper --model model.onnx --output_file output.wav

# Da file di testo
cat testo.txt | piper --model model.onnx --output_file output.wav
```

### Troubleshooting:
- **Qualità audio bassa**: Aumenta epochs o migliora dataset
- **OOM Error**: Riduci batch_size nella configurazione
- **Training interrotto**: Riavvia dalla cella STEP 6 (riprenderà dall'ultimo checkpoint)

### Risorse:
- [Piper GitHub](https://github.com/rhasspy/piper)
- [Piper Documentation](https://github.com/rhasspy/piper/blob/master/README.md)

---

**Buon TTS! 🎤🚀**